<a href="https://colab.research.google.com/github/arkapravag/SNN-Verification/blob/main/LLMs_for_Creating_Software_Final_Project_Final_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import google.generativeai as genai
from google.colab import userdata
import json
import re

In [ ]:
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")

# Setting up the map using the Game-Generator-LLM

In [ ]:
game_generator_prompt = '''Generate a Zork map and specify the winning condition.\n
The zork map should be in the form of a graph. The graphs can have cycles.\n
A player should be able to navigate this map later and find the winning condition. Every room should have  2 or 3 survival objects ,One correct object that counters the enemy, An enemy field for clarity A correct_object field indicating the optimal survival item\n
An example is as follows {
  "map": {
    "Clearing": {
      "description": "You are standing in a clearing. A path leads north, south, and east.",
      "exits": {
        "north": "Forest",
        "south": "Creek",
        "east": "RockyPass"
      },
      "objects": ["Rope", "Torch", "Axe"],
      "enemy": "None",
      "correct_object": "None"
    },
    "Forest": {
      "description": "You are in a dense forest. Sunlight barely penetrates the canopy. A path leads south and west.",
      "exits": {
        "south": "Clearing",
        "west": "HiddenCave"
      },
      "objects": ["Knife", "HealingHerb", "SturdyStick"],
      "enemy": "Giant Spider",
      "correct_object": "Torch"
    },
    "Creek": {
      "description": "You are beside a clear, flowing creek. A path leads north and east.",
      "exits": {
        "north": "Clearing",
        "east": "DarkWoods"
      },
      "objects": ["FishingRod", "Waterskin", "SharpRock"],
      "enemy": "Grizzly Bear",
      "correct_object": "SharpRock"
    },
    "RockyPass": {
      "description": "You are in a rocky pass. The path continues west and north.",
      "exits": {
        "west": "Clearing",
        "north": "AncientRuins"
      },
      "objects": ["Map", "Compass", "Pickaxe"],
      "enemy": "Rock Serpent",
      "correct_object": "Pickaxe"
    },
    "HiddenCave": {
      "description": "You have found a hidden cave. It is damp and dark. A path leads east.",
      "exits": {
        "east": "Forest"
      },
      "objects": ["GoldenIdol", "RustySword", "Tinderbox"],
      "enemy": "Bat Swarm",
      "correct_object": "Tinderbox"
    },
    "DarkWoods": {
      "description": "You are surrounded by dark, looming trees. The path leads west and south.",
      "exits": {
        "west": "Creek",
        "south": "OminousGrave"
      },
      "objects": ["Lantern", "Shovel", "DriedRations"],
      "enemy": "Evil Spirit",
      "correct_object": "Lantern"
    },
    "AncientRuins": {
      "description": "You stand before ancient ruins. The air is thick with mystery. A path leads south and east.",
      "exits": {
        "south": "RockyPass",
        "east": "TreasureChamber"
      },
      "objects": ["Scroll", "JeweledSkull", "StrangeKey"],
      "enemy": "Stone Golem",
      "correct_object": "StrangeKey"
        },
    "OminousGrave": {
      "description": "An ominous grave sits here. You feel a chill in the air. The path leads north.",
      "exits": {
        "north": "DarkWoods"
      },
      "objects": ["SilverCross", "HolyWater", "Bone"],
      "enemy": "Undead Knight",
      "correct_object": "SilverCross"
    },
    "TreasureChamber": {
      "description": "You have found the treasure chamber! Gold and jewels gleam in the light.",
      "exits": {},
      "objects": ["GoldCoins", "Jewels", "TreasureChest"],
      "enemy": "None",
      "correct_object": "None",
    }
  },
  "winning_condition": {
    "location": "TreasureChamber",
    "item_needed": "GoldenIdol"
  },
  "starting_location": "Clearing"
}
Generate the game and its components as output in a JSON format\n'''

In [ ]:
response = model.generate_content(game_generator_prompt)

In [ ]:
txt_response = response.text

In [ ]:
clean_json = txt_response.replace('\n', '')
clean_json = clean_json.strip()
if clean_json.startswith("```json"):
    clean_json = clean_json[len("```json"):].lstrip()
elif clean_json.startswith("```"):
    clean_json = clean_json[len("```"):].lstrip()

if clean_json.endswith("```"):
    clean_json = clean_json[:-3].rstrip()

In [ ]:
# Predefined Map

data = {
  "map": {
    "CrystalBridge": {
      "description": "A shimmering crystal bridge stretches over a vast chasm. Paths lead east and south.",
      "exits": {
        "east": "CastleRuins",
        "south": "Lake"
      },
      "objects": ["Shield", "MagicStone", "Whistle"],
      "enemy": "Air Elemental",
      "correct_object": "MagicStone"
    },
    "CastleRuins": {
      "description": "The remains of an ancient castle loom above. Broken towers cast long shadows. Paths lead west, east, and south.",
      "exits": {
        "west": "CrystalBridge",
        "east": "TreasureChamber",
        "south": "ForestClearing"
      },
      "objects": ["IronSword", "Spellbook", "Candle"],
      "enemy": "Ghost Knight",
      "correct_object": "Spellbook"
    },
    "TreasureChamber": {
      "description": "You've entered the Treasure Chamber. Riches gleam under a beam of light from above. Paths lead west and south.",
      "exits": {
        "west": "CastleRuins",
        "south": "DarkCave"
      },
      "objects": ["Crown", "GoldenIdol", "Gems"],
      "enemy": "None",
      "correct_object": "None"
    },
    "DarkCave": {
      "description": "You are in a pitch-black cave. Moisture drips from the ceiling. Paths lead north and west.",
      "exits": {
        "north": "TreasureChamber",
        "west": "ForestClearing"
      },
      "objects": ["Flint", "Lantern", "OldCoin"],
      "enemy": "Cave Troll",
      "correct_object": "Lantern"
    },
    "ForestClearing": {
      "description": "You find yourself in a quiet forest clearing. Birds chirp in the distance. Paths lead north, west, and east.",
      "exits": {
        "north": "CastleRuins",
        "west": "Lake",
        "east": "DarkCave"
      },
      "objects": ["Bow", "Torch", "Net"],
      "enemy": "Forest Wraith",
      "correct_object": "Torch"
    },
    "Lake": {
      "description": "A calm lake glimmers under the sunlight. Paths lead north and east.",
      "exits": {
        "north": "CrystalBridge",
        "east": "ForestClearing"
      },
      "objects": ["Harpoon", "Waterskin", "Mirror"],
      "enemy": "Water Serpent",
      "correct_object": "Harpoon"
    }
  },
  "winning_condition": {
    "location": "TreasureChamber",
    # "item_needed": "GoldenIdol"
  },
  "starting_location": "CrystalBridge"
}

game_data_copy =  data

In [ ]:
def is_villain_present(game_map, current_location):
    """
    Checks if there's a villain in the current location.

    Args:
        game_map (dict): The entire game map.
        current_location (str): The current location name.

    Returns:
        bool: True if a villain is present, False otherwise.
    """
    location_data = game_map.get(current_location, {})
    enemy = location_data.get("enemy", "None")
    return enemy != "None"

In [ ]:
def get_location_details(location_name):
    game_map = data['map']
    output = ""

    if location_name in game_map:
        location = game_map[location_name]
        output += f"--- Details for '{location_name}' ---\n"
        # output += f"Description: {location['description']}\n"
        output += f"Objects: {location['objects']}\n"
        output += f"Enemy: {location['enemy']}\n"
        output += f"Correct Object: {location['correct_object']}\n"
    else:
        output += f"Location '{location_name}' not found in the map.\n"

    return output

In [ ]:
def update_llm_game_host(data, game_history, data_cur_loc, user_move):
    global llm_game_host
    # Instead of calling llm_game_host, format the string with the new values
    llm_game_host = f'''You are a game host for a text-based adventure game.
    Your task is to guide the player through a series of locations, describe the surroundings, and handle their actions. Here are the rules:

    Describe current location, including its description and the enemy.
    Then, ask the player for their next action.

    If the player looks around, they should see all the items that are present in the room.
    If the player encounters an enemy, describe the encounter and ask the player for their next action.

    Here is the history of the game so far: {game_history}

    Here is the current location and its details: {data_cur_loc}

    The user chooses to {user_move}
    You will act according to the user input and output the result using the details of the current location.

    Describe what happens now.
    If the enemy was successfully countered with the correct object in this round, also output "ENEMY CLEAR"
    '''
    return llm_game_host

  #  If the player wins the game by finding the treasure, congratulate them and end the game.
   # If the player's action is invalid, tell them it's invalid and ask for their next action.

In [ ]:
def update_llm_game_host_for_nav(data, game_history, cur_loc, user_move):
    global llm_game_host

    game_map = data['map']
    output = ""
    location_name = cur_loc
    if location_name in game_map:
        location = game_map[location_name]
        output += f"--- Details for '{location_name}' ---\n"
        output += f"Description: {location['description']}\n"
        output += f"Exits: {location['exits']}\n"
    else:
        output += f"Location '{location_name}' not found in the map.\n"

    data_cur_loc = output

    # Instead of calling llm_game_host, format the string with the new values
    llm_game_host = f'''You are a game host for a text-based adventure game.
    Your task is to guide the player through a series of locations, describe the surroundings, and handle their actions.

    Here is the history of the game so far: {game_history}

    Here is the current location and its details: {data_cur_loc}

    Ask which direction the player wants to go.
    '''
    return llm_game_host

In [ ]:
def cleanse_location(game_data, location_name):
    if location_name in game_data['map']:
        game_data['map'][location_name]['enemy'] = 'None'
        game_data['map'][location_name]['objects'] = []
    return game_data

def next_location(game_data, current_location, direction):
    if current_location in game_data['map'] and direction in game_data['map'][current_location]['exits']:
        return game_data['map'][current_location]['exits'][direction]
    return current_location  # Stay in the same location if invalid direction

def contains_direction(text):
    # Regular expression to match cardinal directions
    pattern = r'\b(east|west|north|south|run)\b'
    match = re.search(pattern, text, re.IGNORECASE)
    return bool(match)

def get_last_few_lines(text):
    lines = text.strip().splitlines()
    return '\n'.join(lines[-10:])


def get_place_by_number(data, number):
    places = list(data['map'].keys())
    if 1 <= number <= len(places):
        return places[number - 1]
    else:
        return "Invalid number"

In [ ]:
def generate_final_act_prompt(game_history, cur_loc):

    final_act_prompt = f'''You are a game host for a text-based adventure game.
    Your task is to guide the player through a series of locations, describe the surroundings, and handle their actions.

    Here is the history of the game so far: {game_history}

    The player has successfully reached the {cur_loc}".

    Give him three options to choose from: a gold, a silver and a bronze key.
    '''

    return final_act_prompt

In [ ]:
def generate_result_of_the_game_prompt(game_history, cur_loc, user_input):

    result_of_the_game_prompt = f'''You are a game host for a text-based adventure game.
        Your task is to guide the player through a series of locations, describe the surroundings, and handle their actions.

        Here is the history of the game so far: {game_history}

        The player has successfully reached the {cur_loc}.

        There were three options to choose from: a gold, a silver and a bronze key.

        The user has chosen {user_input}.

        The correct option was the silver key.

        If the user did not choose the silver key, output a crazy explanation of why his chosen output was wrong and the silver key was right.

        Also say that the game has ended.

    '''
    return result_of_the_game_prompt

In [ ]:
# Initialization of variables
cur_loc = get_place_by_number(data,1) # start from the first location in the map
data_cur_loc = get_location_details(cur_loc)

if is_villain_present(data['map'], cur_loc):
    villain_fight_mode = True
else:
    villain_fight_mode = False
prev_loc = ""
user_move = "No moves yet. The game is starting"
game_history = ""
user_move = "None"
user_direction = 0
move_successful = 0
enemy_just_defeated = False

for i in range(100):

    if villain_fight_mode == True and contains_direction(user_move):
        user_move = "look for any enemies"

    llm_game_host = update_llm_game_host(data, game_history, data_cur_loc, user_move)

    if cur_loc == "TreasureChamber":
        text = model.generate_content(generate_final_act_prompt(game_history, cur_loc)).text
        print()
        print("-----------------------------------------------------------------------")
        print(text)
        user_move = input()
        game_history += "\nGame: "+text
        game_history += "\nPlayer: " + user_move
        text = model.generate_content(generate_result_of_the_game_prompt(game_history, cur_loc, user_move)).text
        print(text)
        print("THE END")
        break

    if villain_fight_mode == True:
        print()
        print("-----------------------------------------------------------------------")
        text = model.generate_content(llm_game_host).text
        if( "ENEMY CLEAR" in text):
            data = cleanse_location(data, cur_loc)
            text = text.replace('ENEMY CLEAR', '')
            villain_fight_mode = False
            enemy_just_defeated = True
        else:
            pass
        print()
        print(text)
        prev_loc = cur_loc

        if enemy_just_defeated:
            pass
        else:
            user_move = input()

        enemy_just_defeated = False

        game_history += "\nGame: "+text
        game_history += "\nPlayer: " + user_move

    else:
        # LLM describing the exits if there is no enemy
        llm_game_host_nav = update_llm_game_host_for_nav(data, game_history, cur_loc, user_move)
        text = model.generate_content(llm_game_host_nav).text
        print()
        print("-----------------------------------------------------------------------")
        print(text)
        user_move = input()
        user_direction = user_move
        if user_direction != 0 and i!=0:
            try:
                user_direction = re.search(r'(north|south|east|west)', user_direction, re.IGNORECASE).group(1)
            except AttributeError:
                print("Invalid move")
                continue
        cur_loc = next_location(data, cur_loc, user_direction)
        if cur_loc == prev_loc:
            print("Invalid! Please try again.")
        else:
          game_history += "\nGame: "+text
          game_history += "\nPlayer: " + user_move
        data_cur_loc = get_location_details(cur_loc)
        if is_villain_present(data['map'], cur_loc):
            villain_fight_mode = True
        else:
            villain_fight_mode = False
        llm_game_host = update_llm_game_host(data, game_history, data_cur_loc, user_move)


    game_history = get_last_few_lines(game_history)


-----------------------------------------------------------------------

You stand on the Crystal Bridge, a shimmering expanse of interwoven crystal strands suspended high above a chasm. The air hums with barely contained energy. Before you swirls an Air Elemental, a sentient vortex of wind and crackling lightning, its form constantly shifting and reforming. It howls, and the bridge vibrates under the force of its gust.

What will you do?



KeyboardInterrupt: Interrupted by user